In [1]:
%load_ext autoreload
%autoreload 2
import os, sys, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

In [2]:
def plot_data(data, x, y, outdir, tag, title, timing=None, smooth=1):
    if smooth > 1:
        """
        smooth data with moving window average.
        that is,
            smoothed_y[t] = average(y[t-k], y[t-k+1], ..., y[t+k-1], y[t+k])
        where the "smooth" param is width of that window (2k+1)
        """
        y = np.ones(smooth)
        for datum in data:
            x = np.asarray(datum[y])
            z = np.ones(len(x))
            smoothed_x = np.convolve(x,y,'same') / np.convolve(z,y,'same')
            datum[y] = smoothed_x
            
    if isinstance(data, list):
        data = pd.concat(data, ignore_index=True)
        if timing:
            data = data[data.timing == timing].drop('timing', axis=1)

    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111)
    sns.set(style="whitegrid", font_scale=1.5)
    sns.set_palette('Set2') # or husl
    if 'timing' in data.columns:
        sns.lineplot(x=x, y=y, ax=ax, data=data, hue=tag, style='timing')
    else:
        sns.lineplot(x=x, y=y, ax=ax, data=data, hue=tag)
    ax.grid(True, alpha=0.8, linestyle=':')
    ax.legend(loc='best').set_draggable(True)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if timing:
        title = f'{title}-{timing}'
    outpath = f'{outdir}/{title}.png'
    ax.set_title(title)
    fig.savefig(outpath)
    fig.show()
    print(f'Plot Path: {outpath}')

def get_datasets(files, tag, condition=None):
    unit = 0
    datasets = []
    for f in files:
        assert f.endswith('log.txt')
        data = pd.read_csv(f, sep='\t')

        data.insert(len(data.columns), tag, condition)

        datasets.append(data)
        unit +=1

    return datasets

In [7]:
import tensorflow as tf
b = 1
u = 3
d = 4
raw_ae = tf.random.normal((b, u, u, d))
mask = tf.cast(tf.random.uniform((b, u), 0, 2, dtype=tf.int32), tf.bool)
print('mask', mask)
mask1 = tf.expand_dims(mask, -2)
mask1 = tf.expand_dims(mask1, -1)
mask2 = tf.expand_dims(mask, -1)
mask2 = tf.expand_dims(mask2, -1)
mask = tf.math.logical_and(mask1, mask2)
raw_ae = tf.where(mask, raw_ae, tf.zeros_like(raw_ae))
print(mask, raw_ae)

mask tf.Tensor([[False  True  True]], shape=(1, 3), dtype=bool)
tf.Tensor(
[[[[False]
   [False]
   [False]]

  [[False]
   [ True]
   [ True]]

  [[False]
   [ True]
   [ True]]]], shape=(1, 3, 3, 1), dtype=bool) tf.Tensor(
[[[[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]

  [[ 0.          0.          0.          0.        ]
   [ 0.5927538   1.1629112  -1.266045   -0.42372763]
   [ 0.171547   -1.2593116  -1.1901214  -0.24145375]]

  [[ 0.          0.          0.          0.        ]
   [-0.05628216 -0.6279982  -1.2477854  -0.7696505 ]
   [ 0.74516904  0.42692438  1.4101542  -1.0267613 ]]]], shape=(1, 3, 3, 4), dtype=float32)


In [36]:
from tensorflow_probability import distributions as tfd
mean = np.random.normal(size=(2, 3))
d = tfd.MultivariateNormalDiag(mean, np.abs(np.random.normal(size=(2, 3))))
print(mean)
d.mean()

[[-0.49860578 -0.11736446  0.50636331]
 [ 0.99319608 -0.5362236  -0.66417662]]


<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[-0.49860578, -0.11736446,  0.50636331],
       [ 0.99319608, -0.5362236 , -0.66417662]])>

In [ ]:
import numpy as np
import random
import gym

from env.func import create_env
config = {
    'env_name': 'grf-11_vs_11_stochastic',
    'seed': 42
}
random.seed(42)
np.random.seed(42)
env = create_env(config)
o, r, d, re = env.reset()
a = env.random_action()
print('action', a)
o, r, d, re = env.step(a)
print('obs', o[0]['obs'][0, 0, 50:60])
a = env.random_action()
print('action', a)
o, r, d, re = env.step(a)
print('obs', o[0]['obs'][0, 0, 50:60])

In [30]:
input_length = 4
x = tf.reshape(tf.range(2*input_length*3), (2, 4, 3))
idx = np.tile(
    np.arange(0, input_length, dtype=np.int32), 
    [input_length, 1]
)
y1 = tf.gather(x, idx, axis=-2)
print(y1)

tf.Tensor(
[[[[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]
   [ 9 10 11]]

  [[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]
   [ 9 10 11]]

  [[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]
   [ 9 10 11]]

  [[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]
   [ 9 10 11]]]


 [[[12 13 14]
   [15 16 17]
   [18 19 20]
   [21 22 23]]

  [[12 13 14]
   [15 16 17]
   [18 19 20]
   [21 22 23]]

  [[12 13 14]
   [15 16 17]
   [18 19 20]
   [21 22 23]]

  [[12 13 14]
   [15 16 17]
   [18 19 20]
   [21 22 23]]]], shape=(2, 4, 4, 3), dtype=int32)


In [33]:
%%timeit
y = tf.expand_dims(x, 1)
y = tf.tile(y, (1, input_length, 1, 1))
# tf.debugging.assert_equal(y, y1)

99.9 µs ± 6.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
%%timeit
y1 = tf.gather(x, idx, axis=-2)

31.9 µs ± 1.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [179]:
def _idx2sp(si, ns):
    """ Convert strategy profile index (si) to a strategy profile """
    sp = []
    for k, _ in enumerate(ns):
        n = np.prod(ns[k+1:], dtype=np.int32)
        i = si // n
        si -= i * n
        sp.append(i)
    return sp

from scipy.linalg import eig
transition = np.array([
    [.5-5e-6, .25, 2.5e-6, .25, 2.5e-6],
    [2.5e-6, 1-7.5e-6-.64e-2, 2.5e-6, 2.5e-6, .64e-2],
    [.25, .25, 0, .25, .25],
    [2.5e-6, .25, 2.5e-6, .75-7.5e-6, 2.5e-6],
    [.25, .14, 2.5e-6, .25, .36-2.5e-6]
])
eps = 1e-2
transition = np.array([
    [.5-2*eps, .25, eps, .25, eps],
    [eps, 1-.64e-2-3*eps, eps, eps, .64e-2],
    [.25, .25, 0, .25, .25],
    [eps, .25, eps, .75-3*eps, eps],
    [.25, .14, eps, .25, .36-eps]
])
print('transition', transition.astype(np.float32), sep='\n')
eig_vals, eig_vecs, reig_vecs = eig(transition, left=True, right=True)
print('eig values', eig_vals, sep='\n')
print('leig vectors', eig_vecs, sep='\n')
print('reig vectors', reig_vecs, sep='\n')
print(eig_vecs.T @ reig_vecs)
mask = abs(eig_vals - 1.) < 1e-10
eig_vecs = eig_vecs[:, mask]
num_stationary_eigenvecs = np.shape(eig_vecs)[1]
if num_stationary_eigenvecs != 1:
    raise ValueError('Expected 1 stationary distribution, but found %d' %
                     num_stationary_eigenvecs)
eig_vecs *= 1. / sum(eig_vecs)

pi = np.squeeze(eig_vecs.real)

# print('eigen vectors', eig_vecs)
rank = np.argsort(pi)[::-1]
# print('pi', pi)
# print('rank', rank)

transition
[[0.48   0.25   0.01   0.25   0.01  ]
 [0.01   0.9636 0.01   0.01   0.0064]
 [0.25   0.25   0.     0.25   0.25  ]
 [0.01   0.25   0.01   0.72   0.01  ]
 [0.25   0.14   0.01   0.25   0.35  ]]
eig values
[-0.01      +0.j  1.        +0.j  0.33894303+0.j  0.47      +0.j
  0.71465697+0.j]
leig vectors
[[-1.27451325e-01  3.40685858e-02 -8.42122012e-01 -7.07106781e-01
   6.72853276e-03]
 [-8.49675503e-02  9.95122638e-01  1.35015231e-01  1.03005446e-16
  -7.13835314e-01]
 [ 8.13834645e-01  1.13573131e-02  1.77027800e-16  7.86424168e-17
  -2.09731072e-17]
 [-4.24837751e-02  9.04579693e-02  2.47248629e-01  7.07106781e-01
   7.00247688e-01]
 [-5.58931994e-01  1.60821211e-02  4.59858152e-01 -4.93527975e-16
   6.85909340e-03]]
reig vectors
[[-0.009998    0.4472136  -0.02484188 -0.38441533  0.4445365 ]
 [-0.009998    0.4472136  -0.01665756  0.0278215  -0.07090326]
 [ 0.99980006  0.4472136   0.5609866  -0.55882322  0.49465702]
 [-0.009998    0.4472136  -0.02484188  0.03923729  0.4445365 ]


In [172]:
from scipy.linalg import svd
eig_vecs, eig_vals, reig_vecs = svd(transition)
print('eig values', eig_vals, sep='\n')
print('leig vectors', eig_vecs, sep='\n')
print('reig vectors', reig_vecs, sep='\n')
print(eig_vecs @ reig_vecs)

eig values
[1.21239455 0.75463785 0.50032083 0.30512244 0.00815111]
leig vectors
[[-0.38725439  0.2374082   0.44065407 -0.76483208  0.12052911]
 [-0.63831854 -0.75921965 -0.07435436  0.05809347  0.08503852]
 [-0.36453393  0.20961168  0.29423372  0.2919742  -0.80706601]
 [-0.46414626  0.46456775 -0.75163029 -0.04768369  0.03902841]
 [-0.30702889  0.32781492  0.38571212  0.569328    0.57040549]]
reig vectors
[[-3.00890052e-01 -7.93513490e-01 -1.48198297e-02 -4.99237477e-01
  -1.74194811e-01]
 [ 3.25144129e-01 -6.06639402e-01  3.58544470e-03  6.89874333e-01
   2.24344558e-01]
 [ 7.46002452e-01 -4.36395883e-02  7.62576243e-06 -5.23199039e-01
   4.09681210e-01]
 [-4.97143876e-01  1.81874388e-02 -6.06623033e-03 -3.15735222e-02
   8.66881478e-01]
 [-8.64756811e-03 -9.47527660e-03  9.99865350e-01 -1.00610178e-02
   1.86991793e-03]]
[[ 0.90163119  0.12898807  0.13174615  0.14949974 -0.36154675]
 [-0.13987684  0.97058252  0.09141173 -0.16888127 -0.03907744]
 [ 0.25916369  0.16222109 -0.80257239 

In [214]:
def positional_encoding(n_pos, size, base=10000):
    denominator = base ** (2 / size)
    pos_encodings = []
    for pos in range(1, n_pos+1):
        pe = []
        for i in range(size):
#             print(i, (np.sin if i % 2 == 0 else np.cos), i//2)
            pe.append((np.sin if i % 2 == 0 else np.cos)(pos/denominator**(i//2)))
        pos_encodings.append(pe)
    return pos_encodings
def positional_encodings(n_pos, size, base=10000):
    denominator = base ** (2 / size)
    pos_encodings = [
        [(np.sin if i % 2 == 0 else np.cos)(pos/denominator**(i//2))
            for i in range(size)]
        for pos in range(1, n_pos+1)]
    return np.array(pos_encodings)
n_units = 11
embed_size = 10
np.testing.assert_allclose(positional_encoding(n_units, embed_size), positional_encodings(n_units, embed_size))
pes = positional_encodings(n_units, embed_size)
idx = np.stack([
    np.concatenate([
        np.arange(i+1, n_units, dtype=np.int32),
        np.arange(0, i, dtype=np.int32), 
    ], -1)
    for i in range(n_units)
])
print(idx)
p1 = pes[idx]
p2 = tf.gather(pes, idx).numpy()
print(p1.shape)
print(p2)
np.testing.assert_allclose(p1, p2)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 110 / 110 (100%)
Max absolute difference: 0.95644914
Max relative difference: 11.1825927
 x: array([[ 8.414710e-01,  5.403023e-01,  1.578266e-01,  9.874668e-01,
         2.511622e-02,  9.996845e-01,  3.981061e-03,  9.999921e-01,
         6.309573e-04,  9.999998e-01],...
 y: array([[ 0.000000e+00,  1.000000e+00,  0.000000e+00,  1.000000e+00,
         0.000000e+00,  1.000000e+00,  0.000000e+00,  1.000000e+00,
         0.000000e+00,  1.000000e+00],...

In [277]:
n_units = 2
embed_size = 3
def positional_encodings(n_pos, size, base=10000):
    denominator = base ** (2 / size)
    pos_encodings = [
        [(np.sin if i % 2 == 0 else np.cos)(pos/denominator**(i//2))
            for i in range(size)]
        for pos in range(1, n_pos+1)]
    return np.array(pos_encodings)

uid = positional_encodings(n_units, embed_size)
idx = np.tile(np.arange(0, n_units, dtype=np.int32), [n_units, 1])
print(uid[idx])

[[[ 0.84147098  0.54030231  0.00215443]
  [ 0.90929743 -0.41614684  0.00430886]]

 [[ 0.84147098  0.54030231  0.00215443]
  [ 0.90929743 -0.41614684  0.00430886]]]


In [292]:
x = tf.zeros((2, 3))
x.shape.ndims

2